In [13]:
import discogs_client
import pandas as pd
import plotly.express as px
from secret import discogs_user_token
import requests
import time
from datetime import datetime
d = discogs_client.Client('ExampleApplication/0.1')


In [3]:
d = discogs_client.Client('my_user_agent/1.0', user_token=discogs_user_token)

In [4]:
me=d.identity()
print(me)

<User 413391 'bat13'>


In [5]:
collection = pd.DataFrame()
title = list()
artist = list()
year = list()
num_for_sale = list()
lowest_price = list()
json_res = list()

# loop to get all data from collection
for item in me.collection_folders[0].releases:
    print(item.release.title)
    title.append(item.release.title)
    artist.append(", ".join(artist.name for artist in item.release.artists))
    year.append(item.release.year)

    response = requests.get(item.release.data['resource_url'])

    try:
        response.raise_for_status()
        json_response = response.json()
    except requests.exceptions.HTTPError as err:
        print(err)
        print(response.text)
        json_response = None

    if 'num_for_sale' in json_response:
        #print(json_response['num_for_sale'])
        num_for_sale.append(json_response['num_for_sale'])
        lowest_price.append(json_response['lowest_price'])
    else:
        #print(json_response)
        num_for_sale.append(0)
        lowest_price.append(0)
    json_res.append(json_response)
    #print(json_response)
    time.sleep(5) # delay to comply with discogs API
    


Infinity Pool
Lux Prima
Milano
Sleep Well Beast
Fordlandia
High Violet
Trouble Will Find Me
Epitaph
Gluey Porch Treatments
The Great Regression
The Big Come Up
Amor Y Muerte En La Tercera Fase
Otros Mundos
Eleven Miles
Satellites
Suave Con La Canoa
System Of A Down
Diabolus In Musica
Mezmerize
Hypnotize
Steal This Album!
God Hates Us All
Il Mio Nome E' Nessuno (Colonna Sonora Originale)
New Bermuda
Unflesh
Selected Ambient Works 85-92
1692
TRST
Self Control
The Desaturating Seven
Four Foot Shack
South Of Reality
Mettal EP
Belfry
Feast For Water
Küken
Louise Attaque
Alquimia Para Principiantes
Sealand
Jupiter Lion
Son Imaginacions Teves
†
Sucks
Fall Off The Apex
Ephemeral Feast
La Jungle
How Did We Get So Dark ?
Royal Blood
Moon
Unfun
Master Of Puppets
Kill 'Em All
Possessor
Hartzazu Koktela Hauekin...
Volatile Times
Lament
Soulfly
3
Revolusongs
Existential Reckoning
Speritismo
Moderat
Orchestra Of Bubbles
Deep Tissue
Silent Shout
Deep Cuts
Speritismo
Hard In Heaven
As The Valley Of Dea

In [7]:
collection['title'] = title
collection['artist'] = artist
collection['year'] = year
collection['num_for_sale'] = num_for_sale
collection['lowest_price'] = lowest_price
collection['json_response'] = json_res
collection

,title,artist,year,num_for_sale,lowest_price,json_response
0,Infinity Pool,When Saints Go Machine,2013,6,22.99,"{'id': 4596009, 'status': 'Accepted', 'year': ..."
1,Lux Prima,"Karen O, Danger Mouse",2019,11,67.46,"{'id': 13381093, 'status': 'Accepted', 'year':..."
2,Milano,"Daniele Luppi, Parquet Courts",2017,35,11.81,"{'id': 11295044, 'status': 'Accepted', 'year':..."
3,Sleep Well Beast,The National,2017,108,20.09,"{'id': 10823938, 'status': 'Accepted', 'year':..."
4,Fordlandia,Jóhann Jóhannsson,2019,45,20.00,"{'id': 14492461, 'status': 'Accepted', 'year':..."
...,...,...,...,...,...,...
495,Default,Atoms For Peace (2),2012,15,4.45,"{'id': 4029978, 'status': 'Accepted', 'year': ..."
496,I Saw Nothing Good So I Left,Bragolin,2023,4,27.11,"{'id': 28045602, 'status': 'Accepted', 'year':..."
497,xx,The XX,2009,4,31.90,"{'id': 1932940, 'status': 'Accepted', 'year': ..."
498,Fantasy Black Channel,Late Of The Pier,2008,9,65.00,"{'id': 1432364, 'status': 'Accepted', 'year': ..."


In [16]:
# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save the dataframe to a pickle file with the timestamp in the name
collection.to_pickle(f'./data/collection_{timestamp}.pkl')
collection.to_pickle(f'./data/collection.pkl')


In [17]:
collection = pd.read_pickle(f'./data/collection.pkl')


In [18]:
collection_by_year = collection.groupby(['year']).agg(
    records = ('title',pd.Series.nunique)
)
collection_by_year.reset_index(inplace=True)
collection_by_year[:10]

,year,records
0,0,8
1,1974,1
2,1975,1
3,1976,1
4,1980,1
5,1984,1
6,1985,2
7,1986,1
8,1987,1
9,1988,3


In [19]:
g = px.bar(collection_by_year[collection_by_year.year>1970], x='year',y='records')
g.show()

In [20]:
release = d.release(1)

response = requests.get(release.data['resource_url'])
response.json()['num_for_sale']
response.json()['lowest_price']


21.6